In [ ]:
"""
    Instruções:
       [v]- Qual a área total desmatada em hectares agosto e setembro de 2022?
       [v]- Qual a área total desmatada em km²(ago/set de 2022) por bioma?
       [v]- Qual a área total desmatada em km² setembro de 2022, por município?
       [v]- Correlação entre as variáveis de população e PIB dos municípios de Minas Gerais e a área desmatada em hectares
"""

In [1]:
import geopandas as gpd
from datetime import datetime

def load_dataframes(populacao_pib_path, focos_desmatamento_path):
    gdf_populacao_pib = gpd.read_file(populacao_pib_path)
    gdf_focos_desmatamento = gpd.read_file(focos_desmatamento_path)

    gdf_focos_desmatamento = gdf_focos_desmatamento.to_crs("EPSG:31983")
    gdf_populacao_pib.crs = "EPSG:31983"

    return gdf_populacao_pib, gdf_focos_desmatamento

def extract_month(data):
    if data is not None:
        try:
            return datetime.strptime(data, '%d/%m/%Y').strftime('%b').lower()
        except ValueError:
            return None
    else:
        return None

def preprocess_dataframes(gdf_populacao_pib, gdf_focos_desmatamento):
    gdf_focos_desmatamento['mes'] = gdf_focos_desmatamento['Data Final'].apply(extract_month)
    gdf_populacao_pib = gdf_populacao_pib.drop(columns=["geometry"])
    gdf_populacao_pib['nome_municipio'] = gdf_populacao_pib['nome_municipio'].replace({
        'AraxÃ¡': 'Araxa',
        'ConceiÃ§Ã£o do Mato Dentro': 'Conceicao do Mato Dentro',
        'DivinÃ³polis': 'Divinapolis',
        'ItajubÃ¡': 'Itajuba',
        'ItaÃºna': 'Itauna',
        'JoÃ£o Monlevade': 'Joao Monlevade',
        'ParÃ¡ de Minas': 'Para de Minas',
        'PatrocÃ­nio': 'Patrocinio',
        'PoÃ§os de Caldas': 'Poco de Caldas',
        'RibeirÃ£o das Neves': 'Ribeirao das Neves',
        'SabarÃ¡': 'Sabara',
        'TimÃ³teo': 'Timoteo',
        'UbÃ¡': 'Ubao',
        'UberlÃ¢ndia': 'Uberlandia',
        'UnaÃ­': 'Unai'
    })

    return gdf_populacao_pib, gdf_focos_desmatamento

def save_to_csv(dataframe, output_csv_path):
    dataframe.to_csv(output_csv_path, index=False)
    print(f"Arquivo CSV salvo em: {output_csv_path}")

populacao_pib_path = "dados/populacao_pib_municipios_mg_ordenado.csv"
focos_desmatamento_path = "dados/focos-desmatamento-mg.geojson"
output_csv_path_pib = "dados/pib.csv"
output_csv_path_agoset = "dados/ago_set.csv"

gdf_populacao_pib, gdf_focos_desmatamento = load_dataframes(populacao_pib_path, focos_desmatamento_path)

gdf_populacao_pib, gdf_focos_desmatamento = preprocess_dataframes(gdf_populacao_pib, gdf_focos_desmatamento)

gdf_agoset_check = gdf_focos_desmatamento[(gdf_focos_desmatamento['mes'] != 'jun') & (gdf_focos_desmatamento['mes'] != 'jul')]

save_to_csv(gdf_populacao_pib, output_csv_path_pib)
save_to_csv(gdf_agoset_check, output_csv_path_agoset)

Arquivo CSV salvo em: dados/pib.csv
Arquivo CSV salvo em: dados/ago_set.csv


In [2]:
import pandas as pd
import numpy as np

def carregar_datasets():
    dataset_pib = pd.read_csv('dados/pib.csv')
    dataset_agoset = pd.read_csv('dados/ago_set.csv')
    return dataset_pib, dataset_agoset

def preencher_valores_nulos(dataset):
    mediana_ogc_fid = dataset["ogc_fid"].median()
    dataset["ogc_fid"].fillna(mediana_ogc_fid, inplace=True)

    dataset['Data Processamento'] = pd.to_datetime(dataset['Data Processamento'], format='%d/%m/%Y %H:%M')
    mediana_data_processamento = dataset['Data Processamento'].median()
    dataset['Data Processamento'].fillna(mediana_data_processamento, inplace=True)

def distribuir_municipio_regiao(regiao, dataset_agoset, dataset_pib):
    municipio_representante = np.random.choice(regioes_municipios[regiao])
    municipio_id = dataset_pib.loc[dataset_pib['nome_municipio'] == municipio_representante, 'municipio_id'].values
    if len(municipio_id) > 0:
        dataset_agoset.loc[dataset_agoset['NuFis'] == regiao, 'municipio_id'] = municipio_id[0]

def adicionar_coluna_municipio_id(dataset_agoset):
    dataset_agoset['municipio_id'] = 0
    
def remover_linhas_nordeste(dataset_agoset):
    dataset_agoset.drop(dataset_agoset[dataset_agoset['NuFis'] == 'Nordeste'].index, inplace=True)

def main(regioes_municipios):
    dataset_pib, dataset_agoset = carregar_datasets()

    preencher_valores_nulos(dataset_agoset)

    adicionar_coluna_municipio_id(dataset_agoset)

    for regiao in regioes_municipios.keys():
        distribuir_municipio_regiao(regiao, dataset_agoset, dataset_pib)
    
    remover_linhas_nordeste(dataset_agoset)
    
    output_csv_path = 'dados/ago_set_relacionamneto.csv'
    
    print(f"Arquivo CSV salvo em: {output_csv_path}")
    
    dataset_agoset.to_csv(output_csv_path, index=False)

if __name__ == "__main__":
    regioes_municipios = {
        'Triângulo Mineiro': ['Uberlandia'],
        'Sul': ['Varginha'],
        'Alto Paranaíba': ['Para de Minas'],
        'Noroeste': ['Unai'],
        'Alto São Francisco': ['Divinapolis'],
        'Norte': ['Montes Claros'],
        'Central Metropolitana': ['Belo Horizonte'],
        'Jequitinhonha': ['Governador Valadares'],
        'Zona da Mata': ['Juiz de Fora'],
        'Leste': ['Ipatinga'],
    }

    main(regioes_municipios)
    

Arquivo CSV salvo em: dados/ago_set_relacionamneto.csv


In [3]:
import pandas as pd


def carregar_datasets():
    populacao_pib_municipios = pd.read_csv('dados/populacao_pib_municipios_mg_ordenado.csv')
    ago_set_relacionamento = pd.read_csv('dados/ago_set_relacionamneto.csv')
    return populacao_pib_municipios, ago_set_relacionamento

def criar_relacionamento_e_remover_geometria(populacao_pib_municipios, ago_set_relacionamento):
    nova_tabela = pd.merge(populacao_pib_municipios, ago_set_relacionamento, on='municipio_id', how='inner')
    nova_tabela.drop('geometria', axis=1, inplace=True)
    return nova_tabela

def salvar_novo_dataframe(dataframe, nome_arquivo):
    dataframe.to_csv(nome_arquivo, index=False)

def main():
    populacao_pib_municipios, ago_set_relacionamento = carregar_datasets()
    
    nova_tabela = criar_relacionamento_e_remover_geometria(populacao_pib_municipios, ago_set_relacionamento)
    
    output_csv_path = 'dados/pib_municipios_meses.csv'
    print(f"Arquivo CSV salvo em: {output_csv_path}")
    salvar_novo_dataframe(nova_tabela, output_csv_path)

if __name__ == "__main__":
    main()

Arquivo CSV salvo em: dados/pib_municipios_meses.csv


In [4]:
import pandas as pd

def carregar_e_ajustar_datas(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo)
    df['Data Inicial'] = pd.to_datetime(df['Data Inicial'], format='%d/%m/%Y')
    df['Data Final'] = pd.to_datetime(df['Data Final'], format='%d/%m/%Y')

    df['pib_nominal'] = df['pib_nominal'].replace('[\.,R$ ]', '', regex=True).astype(float)
    df['populacao'] = df['populacao'].replace('[\.,]', '', regex=True).astype(float)

    return df

def calcular_area_desmatada(df, mes, data_inicial, data_final):
    filtro = (
        (df['mes'] == mes) &
        (df['Data Inicial'] <= pd.to_datetime(data_final)) &
        (df['Data Final'] >= pd.to_datetime(data_inicial))
    )
    area_desmatada = df.loc[filtro, 'area_km2'].sum()
    return area_desmatada

def calcular_area_desmatada_por_bioma(df, mes, data_inicial, data_final):
    filtro = (
        (df['mes'] == mes) &
        (df['Data Inicial'] <= pd.to_datetime(data_final)) &
        (df['Data Final'] >= pd.to_datetime(data_inicial))
    )
    area_por_bioma = df.loc[filtro].groupby('Bioma')['area_km2'].sum()
    return area_por_bioma

def calcular_area_desmatada_por_municipio(df, mes, data_inicial, data_final):
    filtro = (
        (df['mes'] == mes) &
        (df['Data Inicial'] <= pd.to_datetime(data_final)) &
        (df['Data Final'] >= pd.to_datetime(data_inicial))
    )
    area_por_municipio = df.loc[filtro].groupby('nome_municipio')['area_km2'].sum()
    return area_por_municipio

def main():
    pib_municipios_meses = carregar_e_ajustar_datas('dados/pib_municipios_meses.csv')

    area_agosto_mg = calcular_area_desmatada(pib_municipios_meses, 'aug', '2022-08-01', '2022-08-31')

    area_setembro_mg = calcular_area_desmatada(pib_municipios_meses, 'sep', '2022-09-01', '2022-09-30')

    area_por_bioma_agosto = calcular_area_desmatada_por_bioma(pib_municipios_meses, 'aug', '2022-08-01', '2022-08-31')
    area_por_bioma_setembro = calcular_area_desmatada_por_bioma(pib_municipios_meses, 'sep', '2022-09-01', '2022-09-30')

    area_agosto_mg_por_municipio = calcular_area_desmatada_por_municipio(pib_municipios_meses, 'aug', '2022-08-01', '2022-08-31')
    area_setembro_mg_por_municipio = calcular_area_desmatada_por_municipio(pib_municipios_meses, 'sep', '2022-09-01', '2022-09-30')

    print("Área total desmatada em Minas Gerais em agosto de 2022 por município:")
    print(area_agosto_mg_por_municipio)
    print("\nÁrea total desmatada em Minas Gerais em setembro de 2022 por município:")
    print(area_setembro_mg_por_municipio)
    print("Área total desmatada por bioma em agosto:")
    print(area_por_bioma_agosto)
    print("Área total desmatada por bioma em setembro:")
    print(area_por_bioma_setembro)
    print(f"Área total desmatada em Minas Gerais em agosto de 2022: {area_agosto_mg} hectares")
    print(f"Área total desmatada em Minas Gerais em setembro de 2022: {area_setembro_mg} hectares")

    correlacao = pib_municipios_meses[['populacao', 'pib_nominal', 'area_km2']].corr()
    print("\nMatriz de correlação:")
    print(correlacao)
    
    area_agosto_mg_por_municipio.to_csv('dados/area_agosto_mg_por_municipio.csv')
    area_setembro_mg_por_municipio.to_csv('dados/area_setembro_mg_por_municipio.csv')
    area_por_bioma_agosto.to_csv('dados/area_por_bioma_agosto.csv')
    area_por_bioma_setembro.to_csv('dados/area_por_bioma_setembro.csv')
    pd.DataFrame({'area_agosto_mg': [area_agosto_mg], 'area_setembro_mg': [area_setembro_mg]}).to_csv('dados/areas_totais.csv', index=False)
    correlacao.to_csv('dados/correlacao.csv')

if __name__ == "__main__":
    main()

Área total desmatada em Minas Gerais em agosto de 2022 por município:
nome_municipio
Belo Horizonte           12575.366071
Divinópolis              11343.306711
Governador Valadares      4708.185842
Montes Claros           547906.434809
Pará de Minas            17626.842382
Uberlândia              218719.288261
Unaí                    406414.292468
Varginha                  3164.039527
Name: area_km2, dtype: float64

Área total desmatada em Minas Gerais em setembro de 2022 por município:
nome_municipio
Belo Horizonte          1.423002e+04
Divinópolis             2.623140e+04
Governador Valadares    3.507598e+05
Ipatinga                1.681042e+04
Juiz de Fora            6.030468e+04
Montes Claros           1.600746e+06
Pará de Minas           6.885485e+04
Uberlândia              5.488616e+05
Unaí                    1.634124e+06
Varginha                4.508756e+04
Name: area_km2, dtype: float64
Área total desmatada por bioma em agosto:
Bioma
Caatinga          282905.936928
Cerrado    